In [10]:
from ase.io import read
from gGA.data import AtomicData
from gGA.gutz import GhostGutzwiller
from gGA.utils.tools import setup_seed
from gGA.utils.make_kpoints import kmesh_sampling
from gGA.data import block_to_feature
import numpy as np
from gGA.data import _keys

setup_seed(124)
U = 0.5
J = 0.0 * U # 0.25 * U
Up = U - 2*J
Jp = J

t = 0.5
Delta = 0.25

intparams = {"C":[{"U":U,"Up":Up,"J":J, "Jp":Jp}]}


gga = GhostGutzwiller(
    atomic_number=np.array([6]),
    nocc=5,
    basis={"C":[5]},
    idx_intorb={"C":[0]},
    naux=1,
    intparams=intparams,
    nspin=4,
    kBT=0.0002,
    mutol=1e-4,
    natural_orbital=False,
    solver="ED",
    mixer_options={"method": "Linear", "a": 0.3},
    iscomplex=True,
    solver_options={}#{"mfepmin":2000, "channels": 10, "Ptol": 1e-5},
)

onsite = np.zeros((5,5))
onsite[:3,:3] = np.eye(3) * Delta

block = {
    "0_0_0_0_0": onsite,
    
    # 1nn for 0
    "0_0_0_0_1": -t*np.eye(5),
    "0_0_0_1_0": -t*np.eye(5),
    "0_0_1_0_0": -t*np.eye(5),

}

for k in block:
    block[k] = np.kron(block[k], np.eye(2))

for k in block:
    block[k] = np.kron(block[k], np.eye(2))

atomicdata = AtomicData.from_ase(
    read("/nessa/users/zhanghao/dev/Hubbard/gGA/test/C_cube.vasp"),
    r_max=3.1
    )

atoms = atomicdata.to_ase()

atomicdata["kpoint"] = kmesh_sampling([10,10,10], True)
block_to_feature(atomicdata, gga.kinetic.idp_phy, block)
atomicdata = AtomicData.to_AtomicDataDict(atomicdata)
atomicdata = gga.kinetic.idp_phy(atomicdata)

In [ ]:
gga.run(atomicdata, 10, 1e-5)

DM_kin:  [0.16305849 0.16305849 0.47846774 0.47846774 0.50006269 0.50006269
 0.54300158 0.54300158 0.81540707 0.81540707]
 -- Current error: 0.43473
DM_kin:  [0.33525461 0.33525461 0.4999223  0.4999223  0.50005435 0.50005435
 0.52176301 0.52176301 0.64300574 0.64300574]
 -- Current error: 0.36924
DM_kin:  [0.45219265 0.45219265 0.4999843  0.4999843  0.5        0.5
 0.50010308 0.50010308 0.54771998 0.54771998]


KeyError: 'hamiltonians'